In [1]:
from data_extraction import DataExtractor

# Create an instance of the DataExtractor
extractor = DataExtractor()

# Call the method
s3_uri = 's3://data-handling-public/products.csv'
df = extractor.extract_from_s3(s3_uri)

# Display the DataFrame
print(df.head())


   Unnamed: 0                                 product_name product_price  \
0           0  FurReal Dazzlin' Dimples My Playful Dolphin        £39.99   
1           1          Tiffany's World Day Out At The Park        £12.99   
2           2          Tiffany's World Pups Picnic Playset         £7.00   
3           3     Tiffany's World Wildlife Park Adventures        £12.99   
4           4                      Cosatto Cosy Dolls Pram        £30.00   

   weight        category            EAN  date_added  \
0   1.6kg  toys-and-games  7425710935115  2005-12-02   
1  0.48kg  toys-and-games   487128731892  2006-01-09   
2    590g  toys-and-games  1945816904649  1997-03-29   
3    540g  toys-and-games  1569790890899  2013-03-20   
4  1.91kg  toys-and-games  7142740213920  2007-12-23   

                                   uuid          removed product_code  
0  83dc0a69-f96f-4c34-bcb7-928acae19a94  Still_avaliable  R7-3126933h  
1  712254d7-aea7-4310-aff8-8bcdd0aec7ff  Still_avaliable  C2-7

In [2]:
# Display the first few entries in the weight column to understand its current format
print(df['weight'].head())

# Get a broader overview by looking at unique values
unique_weights = df['weight'].unique()
print(unique_weights)

0     1.6kg
1    0.48kg
2      590g
3      540g
4    1.91kg
Name: weight, dtype: object
['1.6kg' '0.48kg' '590g' '540g' '1.91kg' '0.91kg' '0.46kg' '0.38kg'
 '8.981kg' '1.478kg' '1.2g' '0.66kg' '1.8kg' '1.9kg' '1.725kg' '0.54kg'
 '0.322kg' '0.71kg' '0.88kg' '0.67kg' '11.076kg' '4kg' '0.385kg' '1.38kg'
 '2.57kg' '1.35kg' '0.695kg' '1.15kg' '0.98kg' '1.447kg' '1.3625kg'
 '2.25kg' '0.79kg' '0.8kg' '1.08kg' '2.476kg' '0.137kg' '11.5kg' '0.44kg'
 '2.75kg' '0.911kg' '0.33kg' '1kg' '0.5kg' '0.45kg' '0.7kg' '0.41kg'
 '1.3kg' '2kg' '0.34kg' '0.37kg' '0.76kg' '1.18kg' '0.685kg' '1.59kg'
 '1.4kg' '1.66kg' '13.5kg' '0.745kg' '1.44kg' '0.74kg' '0.660kg' '0.419kg'
 '0.418kg' '0.470kg' '0.353kg' '0.350kg' '0.96kg' '1.20kg' '1.21kg'
 '1.02kg' '0.365kg' '0.677kg' '0.55kg' '0.43kg' '0.11kg' '1.23kg' '1.03kg'
 '0.87kg' '0.39kg' '0.35kg' '0.42kg' '0.27kg' '726g' '0.61kg' '0.864kg'
 '0.667kg' '0.63kg' '0.72kg' '0.58kg' '0.627kg' '0.3kg' '0.32kg' '0.01kg'
 '0.650kg' '0.68kg' '0.36kg' '1.395kg' '0.9kg' '0.468

In [3]:
df_copy = df.copy()

In [5]:
import pandas as pd
import re

# Define a function to extract numbers and units
def extract_number_unit(s):
    if pd.isna(s):
        return [None, None]
    # Use a raw string for the regular expression
    parts = re.split(r'(\d+\.?\d*)', s.replace(' ', ''))
    if len(parts) >= 3:
        return [parts[1], ''.join(parts[2:])]
    return [None, None]

# Example usage with DataFrame
data = {
    'weight': ['1.5kg', '300 g', '200ml', '2 liters', '?', '500grams']
}
df = pd.DataFrame(data)

# Apply this function to the 'weight' column
df['weight_split'] = df['weight'].apply(extract_number_unit)
df[['number', 'unit']] = pd.DataFrame(df['weight_split'].tolist(), index=df.index)

# Print the DataFrame to see the extracted parts
print(df[['weight', 'number', 'unit']])


     weight number    unit
0     1.5kg    1.5      kg
1     300 g    300       g
2     200ml    200      ml
3  2 liters      2  liters
4         ?   None    None
5  500grams    500   grams


In [7]:
def normalize_units(unit):
    if unit is None:  # Check if the unit is None and return None immediately if true
        return None
    unit = unit.lower().strip()  # Safely apply string methods now
    if 'kg' in unit or 'kilogram' in unit:
        return 'kg'
    elif 'g' in unit or 'gram' in unit:
        return 'g'
    elif 'ml' in unit or 'milliliter' in unit:
        return 'g'  # treating ml as grams as per the 1:1 volume to mass ratio given
    elif 'liter' in unit:
        return 'liters'  # you might need to decide how to handle liters if they need conversion
    else:
        return None  # for handling unknown or unparsable units


In [8]:
# Apply the adjusted function
df_copy['unit'] = df_copy['unit'].apply(normalize_units)

In [9]:
print(df_copy['unit'].unique())

['kg' 'g' None]


In [10]:
def normalize_units(unit):
    if unit is None:  # Check if the unit is None and return None immediately if true
        return None
    unit = unit.lower().strip()  # Safely apply string methods now
    if 'kg' in unit or 'kilogram' in unit:
        return 'kg'
    elif 'g' in unit or 'gram' in unit:
        return 'g'
    elif 'ml' in unit or 'milliliter' in unit:
        return 'g'  # treating ml as grams as per the 1:1 volume to mass ratio given
    elif 'liter' in unit or 'liters' in unit:
        return 'liters'  # Now correctly handling both 'liter' and 'liters'
    else:
        return None  # for handling unknown or unparsable units


In [11]:
# Create a second copy of the original DataFrame to ensure no carry-over mistakes
df_second_copy = df.copy()

# Apply the corrected normalization function
df_second_copy['unit'] = df_second_copy['unit'].apply(normalize_units)

# Check the results
print(df_second_copy['unit'].unique())


['kg' 'g' 'liters' None]


In [12]:
import pandas as pd
import re

# Assuming 'df' is your original DataFrame
# Create another new copy of the original DataFrame
df_final = df.copy()

# Function to normalize units and convert weights to kg
def process_weight(row):
    if pd.isna(row['weight']):
        return None
    # Extract number and unit using regex
    parts = re.split(r'(\d+\.?\d*)', row['weight'].replace(' ', ''))
    if len(parts) < 3:
        return None
    number = parts[1]
    unit = parts[2].lower().strip()
    
    # Normalize units
    if 'kg' in unit or 'kilogram' in unit:
        unit = 'kg'
    elif 'g' in unit or 'gram' in unit:
        unit = 'g'
    elif 'ml' in unit or 'milliliter' in unit or 'millilitre' in unit:
        unit = 'g'  # Treating ml as grams
    elif 'liter' in unit or 'liters' in unit or 'litre' in unit or 'litres' in unit:
        unit = 'liters'  # This requires decision on conversion
    else:
        return None
    
    # Convert number to float
    try:
        number = float(number)
        if unit == 'g':
            return number / 1000  # Convert grams to kilograms
        elif unit == 'kg':
            return number  # Already in kilograms
        elif unit == 'liters':
            # Assuming 1 liter of water = 1 kg
            return number
    except ValueError:
        return None

# Apply the function to create the weight_kg column
df_final['weight_kg'] = df_final['weight'].apply(process_weight)

# Optionally, drop the original 'weight' column if no longer needed
# df_final.drop('weight', axis=1, inplace=True)

# Display the first few rows to see the final DataFrame
print(df_final[['weight', 'weight_kg']].head())


TypeError: string indices must be integers, not 'str'

In [13]:
def process_weight(row):
    if pd.isna(row['weight']):
        return None
    # Extract number and unit using regex
    parts = re.split(r'(\d+\.?\d*)', row['weight'].replace(' ', ''))
    if len(parts) < 3:  # Ensure there are enough parts to avoid indexing errors
        return None
    number = parts[1]
    unit = ''.join(parts[2:]).lower().strip()  # Join all parts beyond the first split to form the unit string
    
    # Normalize units
    if 'kg' in unit or 'kilogram' in unit:
        unit = 'kg'
    elif 'g' in unit or 'gram' in unit:
        unit = 'g'
    elif 'ml' in unit or 'milliliter' in unit or 'millilitre' in unit:
        unit = 'g'  # Treating ml as grams
    elif 'liter' in unit or 'liters' in unit or 'litre' in unit or 'litres' in unit:
        unit = 'liters'  # Correctly handling all variations of 'liter'
    else:
        return None  # Handle unknown units
    
    # Convert number to float and calculate the weight in kilograms
    try:
        number = float(number)
        if unit == 'g':
            return number / 1000  # Convert grams to kilograms
        elif unit == 'kg':
            return number  # Already in kilograms
        elif unit == 'liters':
            # Assuming 1 liter of water = 1 kg
            return number
    except ValueError:
        return None  # Handle any conversion issues or non-numeric input


In [14]:
# Apply the updated function to create the weight_kg column
df_final['weight_kg'] = df_final['weight'].apply(process_weight)

# Display the first few rows to see the final DataFrame
print(df_final[['weight', 'weight_kg']].head())


TypeError: string indices must be integers, not 'str'

In [15]:
# Create a new copy of the original DataFrame
df_final = df.copy()


In [16]:
import re

# Define a function to extract number and unit
def extract_number_unit(s):
    if pd.isna(s):
        return [None, None]
    parts = re.split(r'(\d+\.?\d*)', s.replace(' ', ''))
    if len(parts) < 3:
        return [None, None]
    number = parts[1]
    unit = ''.join(parts[2:]).lower().strip()
    return [number, unit]

# Apply the function to create separate number and unit columns
df_final[['number', 'unit']] = df_final['weight'].apply(extract_number_unit).apply(pd.Series)


In [17]:
# Function to normalize units
def normalize_units(unit):
    if unit is None:
        return None
    unit = unit.lower().strip()
    if 'kg' in unit or 'kilogram' in unit:
        return 'kg'
    elif 'g' in unit or 'gram' in unit:
        return 'g'
    elif 'ml' in unit or 'milliliter' in unit or 'millilitre' in unit:
        return 'g'
    elif 'liter' in unit or 'liters' in unit or 'litre' in unit or 'litres' in unit:
        return 'liters'
    else:
        return None

# Apply the normalization function
df_final['unit'] = df_final['unit'].apply(normalize_units)


In [18]:
# Function to convert weights to kilograms
def convert_to_kilograms(row):
    try:
        number = float(row['number'])
        unit = row['unit']
        if unit == 'g':
            return number / 1000  # Convert grams to kilograms
        elif unit == 'kg':
            return number  # Already in kilograms
        elif unit == 'liters':
            return number  # Assuming 1 liter = 1 kg
        else:
            return None
    except (TypeError, ValueError):
        return None

# Apply the conversion function
df_final['weight_kg'] = df_final.apply(convert_to_kilograms, axis=1)


In [19]:
# Remove any rows where weight_kg could not be calculated
df_final = df_final[df_final['weight_kg'].notna()]


In [20]:
# Drop the original weight, number, and unit columns if no longer needed
df_final.drop(['weight', 'number', 'unit'], axis=1, inplace=True)


In [21]:
# Rename the weight column to weight_kg
# This step is already achieved during the conversion, so no additional renaming is required.

# Display the first few rows to verify the final DataFrame
print(df_final.head())


   weight_split  weight_kg
0     [1.5, kg]        1.5
1      [300, g]        0.3
2     [200, ml]        0.2
3   [2, liters]        2.0
5  [500, grams]        0.5


In [22]:
# Check the data type of the weight_kg column
print(df_final['weight_kg'].dtype)


float64


In [28]:
from data_extraction import DataExtractor

# Create an instance of the DataExtractor
extractor = DataExtractor()

# Call the method
s3_uri = 's3://data-handling-public/products.csv'
df = extractor.extract_from_s3(s3_uri)

# Display the DataFrame
print(df.head())


   Unnamed: 0                                 product_name product_price  \
0           0  FurReal Dazzlin' Dimples My Playful Dolphin        £39.99   
1           1          Tiffany's World Day Out At The Park        £12.99   
2           2          Tiffany's World Pups Picnic Playset         £7.00   
3           3     Tiffany's World Wildlife Park Adventures        £12.99   
4           4                      Cosatto Cosy Dolls Pram        £30.00   

   weight        category            EAN  date_added  \
0   1.6kg  toys-and-games  7425710935115  2005-12-02   
1  0.48kg  toys-and-games   487128731892  2006-01-09   
2    590g  toys-and-games  1945816904649  1997-03-29   
3    540g  toys-and-games  1569790890899  2013-03-20   
4  1.91kg  toys-and-games  7142740213920  2007-12-23   

                                   uuid          removed product_code  
0  83dc0a69-f96f-4c34-bcb7-928acae19a94  Still_avaliable  R7-3126933h  
1  712254d7-aea7-4310-aff8-8bcdd0aec7ff  Still_avaliable  C2-7

In [39]:
# Check for special characters in product names
special_characters = df['product_name'].apply(lambda x: re.findall(r'[^a-zA-Z0-9 ]', x))
print("Special characters found in product names:", special_characters.explode().value_counts())


TypeError: expected string or bytes-like object, got 'float'

In [41]:
# Display the first few rows of the product_price column
print(df['product_price'].head(50))


0     £39.99
1     £12.99
2      £7.00
3     £12.99
4     £30.00
5     £12.99
6      £7.00
7     £12.99
8     £12.99
9     £89.99
10    £24.99
11    £24.99
12    £24.99
13    £12.99
14    £12.99
15    £22.00
16    £22.00
17    £20.00
18    £16.99
19    £12.99
20    £39.99
21    £12.99
22    £12.99
23    £12.99
24    £25.00
25    £45.00
26    £24.99
27    £12.99
28    £18.99
29    £12.99
30    £29.99
31    £12.99
32    £12.99
33    £15.99
34    £30.00
35    £12.99
36    £12.99
37    £19.99
38    £12.99
39    £12.99
40    £12.99
41    £25.00
42    £45.00
43    £24.99
44    £12.99
45    £18.99
46    £12.99
47    £29.99
48    £12.99
49    £12.99
Name: product_price, dtype: object


In [43]:
# Check the data type of the product_price column
print("Data type of product_price:", df['product_price'].dtype)


Data type of product_price: object


In [44]:
# Get an overview of unique values in the product_price column
unique_product_prices = df['product_price'].unique()
print("Unique product prices:", unique_product_prices)
print("Number of unique product prices:", len(unique_product_prices))


Unique product prices: ['£39.99' '£12.99' '£7.00' '£30.00' '£89.99' '£24.99' '£22.00' '£20.00'
 '£16.99' '£25.00' '£45.00' '£18.99' '£29.99' '£15.99' '£19.99' '£11.99'
 '£14.99' '£32.99' '£120.00' '£10.00' '£21.99' '£13.99' '£30.99' '£270.00'
 '£40.99' '£42.99' '£71.99' '£34.99' '£52.99' '£13.00' '£2.25' '£3.00'
 '£3.50' '£4.00' '£26.99' '£17.99' '£31.99' '£44.99' '£27.99' '£33.49'
 '£23.49' '£15.00' '£22.99' '£6.99' '£12.00' '£5.99' '£3.99' '£50.00'
 '£49.99' '£36.49' nan '£1.50' '£2.50' '£8.00' '£2.99' '£4.49' '£5.49'
 '£4.35' '£2.75' '£3.69' '£6.49' '£9.49' '£7.99' '£1.29' '£2.19' '£2.69'
 '£0.39' '£2.00' '£1.69' '£1.39' '£1.99' '£3.25' '£4.29' '£4.69' '£3.49'
 '£10.99' '£1.15' '£35.00' '£38.00' '£18.00' '£32.00' '£14.00' '£6.00'
 '£17.00' '£16.00' '£5.00' '£40.00' '£65.00' '£70.00' '£80.00' '£60.00'
 '£99.00' '£110.00' '£100.00' '£9.00' '£8.99' 'XCD69KUI0K' '£55.00'
 '£46.00' '£34.00' '£85.00' '£9.99' 'N9D2BZQX63' '£24.00' '£28.00'
 'ODPMASE7V7' '£49.00' '£59.00' '£1.00' '£1.49' '£

In [45]:
# Check for any null values in the product_price column
null_product_prices = df['product_price'].isnull().sum()
print("Number of null product prices:", null_product_prices)


Number of null product prices: 4


In [46]:
# Check for product prices that do not start with '£'
non_pound_symbols = df['product_price'].apply(lambda x: not str(x).startswith('£'))
print("Number of product prices without '£' symbol:", non_pound_symbols.sum())
print("Examples of product prices without '£' symbol:\n", df[non_pound_symbols]['product_price'].unique())


Number of product prices without '£' symbol: 7
Examples of product prices without '£' symbol:
 [nan 'XCD69KUI0K' 'N9D2BZQX63' 'ODPMASE7V7']


In [48]:
# Display the first few rows of the category column
print(df['category'].head())


0    toys-and-games
1    toys-and-games
2    toys-and-games
3    toys-and-games
4    toys-and-games
Name: category, dtype: object


In [49]:
# Get unique values in the category column
unique_categories = df['category'].unique()
print("Unique categories:", unique_categories)
print("Number of unique categories:", len(unique_categories))


Unique categories: ['toys-and-games' 'sports-and-leisure' nan 'pets' 'homeware' 'S1YB74MLMJ'
 'C3NCA2CL35' 'WVPMHZP59U' 'health-and-beauty' 'food-and-drink' 'diy']
Number of unique categories: 11


In [51]:
# Get the count of each unique value in the removed column
removed_counts = df['removed'].value_counts()
print("Counts of unique values in 'removed' column:\n", removed_counts)


Counts of unique values in 'removed' column:
 removed
Still_avaliable    1752
Removed              94
T3QRRH7SRP            1
BPSADIOQOK            1
H5N71TV8AY            1
Name: count, dtype: int64


In [55]:
# Ensure the product_code column is treated as a string
df['product_code'] = df['product_code'].astype(str)


In [56]:
# Define a regular expression pattern for the valid format
pattern = r'^[A-Za-z0-9]{2}-[A-Za-z0-9]{8}$'


In [57]:
# Identify invalid product_code entries
invalid_product_codes = df[~df['product_code'].str.match(pattern)]

# Display the invalid entries
print("Invalid product_code entries:")
print(invalid_product_codes['product_code'])
print("Number of invalid entries:", len(invalid_product_codes))


Invalid product_code entries:
10      g5-411324A
19       w1-22434p
25       w4-86578A
28       k6-93543H
38      z8-257244T
           ...    
1797    v0-697720L
1805    U9-412268j
1807    K6-678521h
1816    b3-038131J
1847    s1-001714d
Name: product_code, Length: 195, dtype: object
Number of invalid entries: 195


In [58]:
# Assuming the DataExtractor class is available in the same environment
from data_extraction import DataExtractor

# Create an instance of DataExtractor
extractor = DataExtractor()

# S3 URI
s3_uri = 's3://data-handling-public/products.csv'

# Retrieve data from S3
df = extractor.extract_from_s3(s3_uri)

# Display the first few rows of the dataframe to inspect it before cleaning
print("Before cleaning:")
print(df.head())


Before cleaning:
   Unnamed: 0                                 product_name product_price  \
0           0  FurReal Dazzlin' Dimples My Playful Dolphin        £39.99   
1           1          Tiffany's World Day Out At The Park        £12.99   
2           2          Tiffany's World Pups Picnic Playset         £7.00   
3           3     Tiffany's World Wildlife Park Adventures        £12.99   
4           4                      Cosatto Cosy Dolls Pram        £30.00   

   weight        category            EAN  date_added  \
0   1.6kg  toys-and-games  7425710935115  2005-12-02   
1  0.48kg  toys-and-games   487128731892  2006-01-09   
2    590g  toys-and-games  1945816904649  1997-03-29   
3    540g  toys-and-games  1569790890899  2013-03-20   
4  1.91kg  toys-and-games  7142740213920  2007-12-23   

                                   uuid          removed product_code  
0  83dc0a69-f96f-4c34-bcb7-928acae19a94  Still_avaliable  R7-3126933h  
1  712254d7-aea7-4310-aff8-8bcdd0aec7ff  Stil

In [59]:
# Assuming the DataCleaning class is available in the same environment
from data_cleaning import DataCleaning

# Create an instance of DataCleaning
cleaner = DataCleaning()

# Apply the clean_product_data method to the dataframe
df_cleaned = cleaner.clean_product_data(df)

# Display the first few rows of the cleaned dataframe to inspect the changes
print("After cleaning:")
print(df_cleaned.head())


After cleaning:
   Unnamed: 0                                 product_name  product_price_gbp  \
0           0  FurReal Dazzlin' Dimples My Playful Dolphin              39.99   
1           1          Tiffany's World Day Out At The Park              12.99   
2           2          Tiffany's World Pups Picnic Playset               7.00   
3           3     Tiffany's World Wildlife Park Adventures              12.99   
4           4                      Cosatto Cosy Dolls Pram              30.00   

         category            EAN  date_added  \
0  toys-and-games  7425710935115  2005-12-02   
1  toys-and-games   487128731892  2006-01-09   
2  toys-and-games  1945816904649  1997-03-29   
3  toys-and-games  1569790890899  2013-03-20   
4  toys-and-games  7142740213920  2007-12-23   

                                   uuid          removed product_code  \
0  83dc0a69-f96f-4c34-bcb7-928acae19a94  Still_avaliable  R7-3126933h   
1  712254d7-aea7-4310-aff8-8bcdd0aec7ff  Still_avaliable  C2-7

In [67]:
# Display data types before cleaning
print("\nData types before cleaning:")
print(df_cleaned.dtypes)


Data types before cleaning:
Unnamed: 0             int64
product_name          object
product_price_gbp    float64
category              object
EAN                   object
date_added            object
uuid                  object
removed               object
product_code          object
weight_kg            float64
dtype: object


In [66]:
# Check for any non-numeric values in the product_price column
non_numeric_values = df_cleaned[~df_cleaned['product_price_gbp'].str.replace('.', '').str.isnumeric()]
print("Non-numeric values in product_price column:")
print(non_numeric_values['product_price_gbp'])


AttributeError: Can only use .str accessor with string values!

In [62]:
# Ensure all entries in the product_price column are strings
df['product_price'] = df['product_price'].astype(str)


In [63]:
# Check for any non-numeric values in the product_price column
non_numeric_values = df[~df['product_price'].str.replace('.', '', regex=False).str.isnumeric()]
print("Non-numeric values in product_price column:")
print(non_numeric_values['product_price'])


Non-numeric values in product_price column:
0       £39.99
1       £12.99
2        £7.00
3       £12.99
4       £30.00
         ...  
1848    £15.00
1849    £15.00
1850    £18.00
1851    £15.00
1852    £69.00
Name: product_price, Length: 1853, dtype: object
